Long-term create a jupyter kernel for any REPL interface

https://github.com/n-riesco/nel/blob/master/lib/nel.js

https://github.com/n-riesco/jp-kernel/blob/master/lib/jp-kernel.js

https://github.com/takluyver/bash_kernel/blob/master/bash_kernel/kernel.py

https://github.com/n-riesco/ijavascript/blob/d459956f76a22f9ec89937e02645ee7555f92d2b/lib/kernel.js

https://github.com/notablemind/jupyter-nodejs/blob/1388e1957792ef167ccfc7a9cf6c64c3950ded76/lib/kernel.js

https://github.com/JuliaLang/IJulia.jl

https://github.com/n-riesco/jp-babel/blob/master/lib/kernel.js

looks like it is just passed to the command line in rust

https://github.com/google/evcxr/blob/602db3ef52bfddeb34608877bd72b9d0d112fa26/evcxr/src/module.rs


Fix ijavascript not showing filename

https://nodejs.org/api/vm.html#vm_class_vm_script

Fix the problem with ijavascript not supporting %%

Go through this list and find demo code for every language and make sure it works with our REPL, minimize dependencies

https://github.com/jupyter/jupyter/wiki/Jupyter-kernels

https://jupyter-client.readthedocs.io/en/stable/kernels.html

https://jupyter-client.readthedocs.io/en/stable/wrapperkernels.html


Add HTML, CSS/SCSS kernels with some sort of visual output specially make for the file type

Create a kernels notebook




TODO: implement both meta kernels in python and socket based kernels (through library include in another language or through command like repl interface), translate zmq to every language

TODO: use syntax highlighter to "train" a transpiler to write socket based kernels in any language, at least 
BROWSER C# GO JAVA NODE.JS PHP PYTHON RUBY from Google

TODO: make a processing kernel https://github.com/processing/p5.js/wiki/p5.js,-node.js,-socket.io


In [5]:
var importer = require('../Core');
var {setupSockets, installKernel} = importer.import('bind to jupyter zmq sockets');

var PACKAGE_VERSION = require('../package.json').version;

function getVersion(str) {
    return str.split('.').map(v => parseInt(v, 10)).join('.')
}

var kernelJsonTemplate = (kernel_json) => ({
    argv,
    display_name,
    language,
    metadata,
    env,
    interrupt_mode
} = kernel_json)

var kernelMetaTemplate = (meta_kernel) => ({
    do_execute,
    do_complete,
    do_inspect,
    do_history,
    do_is_complete,
    do_shutdown
} = meta_kernel)

var languageTemplate = (language_info) => ({
    mimetype,
    name,
    file_extension,
    version,
    pygments_lexer,
    codemirror_mode,
    nbconvert_exporter
} = language_info)

var kernelInfoTemplate = (kernel_info) => ({
    protocol_version,
    implementation,
    implementation_version,
    banner,
    language_info,
    help_links
} = kernel_info)

var kernelSessionTemplate = (session) => ({
    init,
    message,
    respond,
    display,
    execute_request,
    inspect_request,
    complete_request,
    history_request,
    kernel_info_request,
    is_complete_request,
    connect_request,
    comm_info_request,
    kernel_info_request,
    shutdown_request,
    interrupt_request,
    input_request
} = session)

/*
var transpile = require("@babel/core").transform.bind({
    presets: [
        [require.resolve("@babel/preset-env"), {
            loose: true,
            targets: {node: true},
        }],
    ],
});
*/

var kernelTemplate = ({
    kernel_info,
    meta_kernel,
    language_info,
    session,
    install
}) => ({
    ...kernelInfoTemplate({...kernel_info, language_info}),
    ...kernelMetaTemplate(meta_kernel),
    language_info: languageTemplate(language_info),
    session: kernelSessionTemplate(session),
    bind_sockets,
    install
})

var kernelJsonTemplate = ({path, name, language}) => ({
    argv: ['python', '-m', path, '-f', '{connection_file}'],
    display_name: display_name,
    language: language
})

var count = 0;
function javascriptTemplate(config) {
    return kernelTemplate({
        kernel_info: {
            protocol_version: 5.1,
            implementation: 'javascript',
            implementation_version: PACKAGE_VERSION,
            banner: 'NodeJS',
            // TODO: automatically create this from installation intructions
            help_links: ['https://nodejs.org']
        },
        language_info: {
            name: 'javascript',
            version: getVersion(process.versions.node),
            file_extension: '.js',
            mimetype: 'application/javascript'
        },
        session: {
            init: (session) => {
                var session = require("child_process")
                    .spawn(process.argv[0], ['--eval', meta_kernel], {
                        cwd: config.cwd,
                        stdio: [process.stdin, process.stdout, process.stderr, "ipc"]
                    })
                session.on('message', session.message);
                return setupSockets()
                    .then(sockets => {})
            },
            message: (respond, request) => {
                if(request.clear) respond({clear_output: request.clear})
                if(request.input) respond({input_request: request.input})
            },
            respond: (respond, request) => respond('status', {
                
            }),
            display: (respond, request) => {
                respond(request.display_id
                        ? {update_display_data: {metadata: {}, data: update.mime, transient: update.display_id}}
                        : {display_data: {metadata: {}, data: update.mime}})
            },
            input_request: (respond, request) => {
                //this.onReplies[response.header.msg_id] = onReply;
            },
            shutdown_request: (respond, request, kernel, session) => {
                respond({status: {execution_state: 'busy'}});
                return Promise.resolve()
                    .then(() => session.kill('SIGTERM'))
                    .then(() => request.content.restart
                          ? kernel.init()
                          : void 0)
                    .then(() => respond({shutdown_reply: request.content}))
                    .then(() => respond({status: {execution_state: 'idle'}}))
            },
            kernel_info_request: (respond, request, kernel) => {
                respond({status: {execution_state: 'busy'}});
                respond({kernel_info_reply: kernelInfoTemplate(kernel_info)})
                respond({status: {execution_state: 'idle'}});
            },
            is_complete_request: (respond, request, kernel) => {
                respond({status: {execution_state: 'busy'}});
                respond({is_complete_reply: {
                    status: kernel.do_is_complete(request, session)
                    ? 'complete': 'incomplete',
                    indent: ''
                }});
                respond({status: {execution_state: 'idle'}});
            },
            execute_request: (respond, request, kernel) => {
                var executionCount = ++count;
                respond({status: {execution_state: 'busy'}});
                respond({execute_input: {
                    execution_count: executionCount,
                    code: request.content.code,
                }})
                var result;
                return Promise.resolve()
                    .then(() => kernel.do_execute(request))
                    .then(r => result = r)
                    .then(() => respond({execute_reply: {
                        status: 'ok',
                        execution_count: executionCount,
                        payload: [], // TODO(NR) not implemented,
                        user_expressions: {}, // TODO(NR) not implemented,
                    }}))
                    .then(() => respond({status: {execution_state: 'idle'}}))
                    .then(() => !result.execute_result.data
                          ? void 0
                          : respond({execute_result: {
                            execution_count: executionCount,
                            data: result.mime,
                            metadata: {},
                        }}))
            },
            complete_request: (respond, request, kernel) => {
                respond({status: {execution_state: 'busy'}});
                var result;
                return Promise.resolve()
                    .then(() => kernel.do_complete(request))
                    .then(r => result = r)
                    .then(() => respond({status: {execution_state: 'idle'}}))
                    .then(() => respond({complete_reply: {
                        matches: result.completion.list,
                        cursor_start: result.completion.cursorStart,
                        cursor_end: result.completion.cursorEnd,
                        status: "ok",
                    }}))
            },
            history_request: (respond, request, kernel) => {
                respond({status: {execution_state: 'busy'}});
                var result;
                return Promise.resolve()
                    .then(() => kernel.do_history(request))
                    .then(r => result = r)
                    .then(() => respond({status: {execution_state: 'idle'}}))
                    .then(() => respond({history_reply: {
                        history: [] // TODO
                    }}))
            },
            inspect_request: (respond, request, kernel) => {
                var executionCount = ++count;
                respond({status: {execution_state: 'busy'}});
                var result;
                return Promise.resolve()
                    .then(() => kernel.do_inspect(request))
                    .then(r => result = r)
                    .then(() => respond({status: {execution_state: 'idle'}}))
                    .then(() => respond({inspect_reply: {
                        found: true,
                        data: {
                            /*
        if (result.inspection) {
            docstring = (
                result.inspection.type + ": " + result.inspection.string
            );
        }

        if (result.doc) {
            docstring = result.doc.usage;
            if (result.doc.usage) {
                docstring += "\n\n" + result.doc.description;
            }
        }*/
                            'text/plain': docstring,
                            'text/html': `<pre>${docstring}</pre>`,
                        },
                        metadata: {},
                        status: "ok",
                    }}))
                    .catch(e => respond({inspect_reply: {
                        status: "error",
                        execution_count: executionCount,
                        ename: result.error.ename,
                        evalue: result.error.evalue,
                        traceback: result.error.traceback,
                    }}))
            }
        },
        meta_kernel: {
            do_init: (request, kernel) => {
                process.on('message', (message) => kernel[message[0]].apply(kernel, message[1]))
            },
            do_execute: (code) => new require('vm').Script(transpile(code)),
            do_shutdown: () => {},
            do_is_complete: (request, kernel) => {
                try {
                    kernel.do_execute(request);
                    return true;
                } catch {
                    return false;
                }
            },
            do_complete: (request) => {
                // TODO: call lang-server code
                throw new Error('not implemented!');
            },
            do_history: (request) => {
                throw new Error('not implemented!');
            },
            do_inspect: (request) => {
                // TODO: call lang-server code
                throw new Error('not implemented!');
            }
        },
        install: installKernel
    })
}

console.log(javascriptTemplate());



reading notebook /Users/briancullinan/jupyter_ops/Core/kernels.ipynb
{ protocol_version: 5.1,
  implementation: 'javascript',
  implementation_version: undefined,
  banner: 'NodeJS',
  help_links: [ 'https://nodejs.org' ],
  language_info:
   { name: 'javascript',
     version: '10.16.0',
     file_extension: '.js',
     mimetype: 'application/javascript' },
  do_init: [Function: do_init],
  do_execute: [Function: do_execute],
  do_shutdown: [Function: do_shutdown],
  do_is_complete: [Function: do_is_complete],
  do_complete: [Function: do_complete],
  do_history: [Function: do_history],
  do_inspect: [Function: do_inspect],
  session:
   { init: [Function: init],
     message: [Function: message],
     respond: [Function: respond],
     display: [Function: display],
     input_request: [Function: input_request],
     shutdown_request: [Function: shutdown_request],
     kernel_info_request: [Function: kernel_info_request],
     is_complete_request: [Function: is_complete_request],
    

bind to jupyter zmq sockets?


In [ ]:
var zmq = require("jmp").zmq;
var util = require('util');

function setupSockets(config) {
  const sockets = {
    control: {
      port: config.control_port,
      type: 'xrep',
    },
    shell: {
      port: config.shell_port,
      type: 'xrep',
    },
    stdin: {
      port: config.stdin_port,
      type: 'router',
    },
    iopub: {
      port: config.iopub_port,
      type: 'pub',
    },
    heartbeat: {
      port: config.hb_port,
      type: 'rep',
    }
  }
  return Promise.all(sockets.map(s => setupSocket.apply(null, s, config)))
}

function setupSocket(config, general) {
  const sock = zmq.socket(config.type);
  const addr = general.transport + '://' + general.ip + ':' + config.port
  return util.promisify(sock.bind.bind(sock))(addr)
}

function installKernel(path) {
    require('fs').writeFileSync(path.join(process.cwd, 'kernel.json'),
                                          JSON.stringify(kernelJsonTemplate({
        display_name: 'Node JS',
        path: path,
        language: 'javascript'
    }), null, 4));
    require('child_process').exec('kernelspec install --replace ./kernel.json', {cwd: process.cwd});
}

module.exports = {
    setupSockets,
    installKernel
};
